# Perkenalan

Nama    : Fahmi 

Batch   : RMT-034

Objective   : Notebook ini berisi skrip dan analisis untuk melakukan validasi data menggunakan Great Expectations dari Clean Data yang sudah diolah.

# Instantiate Data Context

Saya melakukan great expectation dari clean data untuk memastikan apakah data tersebut sudah layak dan sesuai harapan saya maupun perusahaan.

Nah, pertama-tama, saya akan menginisialisasi data context untuk penerapan Great Expectasion yang akan dilakukan selanjutnya.

In [17]:
# Create a data context

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

# Connect to A `Datasource`

Setelah data context diinisialisasi, saya akan mendefinisikan datasource yang bersumber dari Data Clean (data yang sudah diolah) yang path-nya berasal dari komputer lokal.

In [20]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'P2M3_fahmi_data_clean_v5.csv'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'data-clean-fahmi'
path_to_data = 'C:\\Users\Lenovo\Documents\codingan\project_m3\dags\P2M3_fahmi_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# Create an Expectation Suite

Selanjutnya, saya membuat Expectation Suite yang merupakan kumpulan dari kombinasi berbagai Expectation tetang dataset yang digunakan harus seperti apa. Langkah ini termasuk mencakup pendefinisian validator yang dipakai.

In [21]:
# Creat an expectation suite
expectation_suite_name = 'expectation-sales-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,location,total_land_area,number_of_cows,farm_size,date,product_id,product_name,brand,quantity,price_per_unit,...,expiration_date,quantity_sold,price_per_unit_sold,approx_total_revenue,customer_location,sales_channel,quantity_in_stock,minimum_stock_threshold,reorder_quantity,unique_id
0,Telangana,310.84,96,Medium,2022-02-17,5,Ice Cream,Dodla Dairy,222.40,85.72,...,2022-01-21,7,82.24,575.68,Madhya Pradesh,Wholesale,215,19.55,64.03,TMFMWDI572811
1,Uttar Pradesh,19.19,44,Large,2021-12-01,1,Milk,Amul,687.48,42.61,...,2021-10-25,558,39.24,21895.92,Kerala,Wholesale,129,43.17,181.10,UKTLWAM151442
2,Tamil Nadu,581.69,24,Medium,2022-02-28,4,Yogurt,Dodla Dairy,503.48,36.50,...,2022-02-13,256,33.81,8655.36,Madhya Pradesh,Online,247,15.10,140.83,TMRMODY422311
3,Telangana,908.00,89,Small,2019-06-09,3,Cheese,Britannia Industries,823.36,26.52,...,2019-07-26,601,28.92,17380.92,Rajasthan,Online,222,74.50,57.68,TRFSOBC362272
4,Maharashtra,861.95,21,Medium,2020-12-14,8,Buttermilk,Mother Dairy,147.77,83.85,...,2020-10-28,145,83.07,12045.15,Jharkhand,Retail,2,76.02,33.40,MJRMRMB812871


## Expectation 1

Ekspektasi yang paling pertama saya, seluruh values di kolom `unique_id` harus unik. Untuk memvalidasinya, saya akan menjalankan kode berikut.

In [22]:
# Column `unique_id` must be unique

validator.expect_column_values_to_be_unique('unique_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 4325,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Success : true. Jadi, kolom `unique_id` benar-benar sesuai harapan atau ekspektasi. Tidak ada duplikat dari setiap values pada kolom tersebut.

## Expectation 2 

Setelah itu, saya ingin mengecek values (nilai-nilai) yang terdapat pada kolom number_of_cows. Apakah sesuai harapan/eskpektasi bahwa itu rentang (range)-nya tidak kurang dari 10 dan tak lebih dari 100. Ini dilakukan untuk menghindari kesalahan penginputan ke kolom itu yang mengganggu kinerja analisis kemudian.

In [23]:
# Column `number_of_cows` must be between 1-10

validator.expect_column_values_to_be_between('number_of_cows', 10, 100)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 4325,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Walhasil, success: true. Jadi, sesuai harapan juga karena memang tidak ada values pada kolom `number_of_cows` yang berada di luar range 10-100.

## Expectation 3

Ekspektasi ketiga, values yang terdapat pada kolom `sales_channel` harus hanya mencakup 3 value tertentu, yakni 'Retail', 'Wholesale', dan 'Online'.

In [24]:
# Column `sales_channel` must contain one of the following 3 things :
# Retail
# Wholesale
# Online

validator.expect_column_values_to_be_in_set('sales_channel', ['Retail', 'Wholesale', 'Online'])


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 4325,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasilnya true. Jadi, tidak ada values pada kolom `sales_channel` yang tidak termasuk 3 values tadi.

## Expectation 4

Ekspektasi lainnya, kolom `quantity_sold` saya ingin seluruh values-nya bertipe data integer karena aneh jika jumlah barang yang terjual berbilangan desimal.

In [25]:
# Column `quantity_sold` must in form of integer or float

validator.expect_column_values_to_be_in_type_list('quantity_sold', ['int64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Ekspektasi terpenuhi juga. Jadi, seluruh nilai yang terdapat pada kolom `quantity_sold` bertipe integer dan tak ada yang float (desimal).

## Exceptation 5

Ekspektasi ke-5, saya ingin kolom `expiration_date` lebih besar (dalam konteks ini, tanggalnya sesudah) dari `production_date`. Itu akan aneh jika ada tanggal exspired yang mendahului tanggal produksi. 

In [26]:
# Column 'expiration_date' must to be greater than column 'production_date'

validator.expect_column_pair_values_A_To_Be_Greater_Than_B('expiration_date', 'production_date', or_equal=True)

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 4325,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasilnya true juga. Itu berarti seluruh values pada kolom tersebut sudah sesuai harapan. Tidak ada values pada `expiration_date` yang lebih awal dari `production_date`.

## Expectation 6

Ekspektasi ini mirip dengan ekspektasi no 2 di atas. Namun, ada sedikit perbedaan pada cakupan validasinya. Ekspektasi no 2 memeriksa seluruh kolom, sedangkan ekspektasi pada no 6 ini hanya memeriksa nilai minimum dalam 1 kolom saja.   

In [27]:
# Column `minimum_stock_threshold` must not below 10

validator.expect_column_min_to_be_between('minimum_stock_threshold', min_value=10)


Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 10.02
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasilnya, true. Itu berarti nilai minimum pada kolom `minimum_stock_threshold` tidak di bawah 10 karena jumlah ambang batas stok minimum terendah yang ditunjukkan pada kolom itu masih bernilai 10.02.

## Exceptation 7

Ekspektasi terakhir, saya berasumsi bahwa hingga saat ini pola pengelolaan susu (storage_condition) yang paling umum digunakan dan disukai oleh konsumen ialah yang dibekukan (Refrigerated). Adapun saya melakukan ekspektasi ini untuk mengecek kevalidan data sekaligus mengetahui apakah asumsi saya masih bertahan hingga ke depannya.

In [28]:
# validasi apakah benar bahwa nilai yang paling sering muncul di kolom 'storage_condition' sesuai yang diharapkan

validator.expect_column_most_common_value_to_be_in_set(
    column="storage_condition", 
    value_set=['Refrigerated'], 
    ties_okay=True
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "Refrigerated"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasilnya ternyata benar sesuai dugaan. Nilai paling banyak yang muncul (modus) pada kolom storage_condition, yakni Refrigerated. 

Setelah seluruh ekspektasi terpenuhi, saya akan menyimpan expectation suite.

In [29]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

# Checkpoint

Di sini, saya akan membuat checkpoint yang nantinya bisa dijalankan untuk memvalidsi batch secara berkala.

In [30]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [31]:
# Run a checkpoint

checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/32 [00:00<?, ?it/s]

# Data Docs

Berikutnya, saya akan membuat data docs yang nantinya diubah menjadi HTML files dan bisa di brwoser.

In [32]:
# Build data docs

context.build_data_docs()

{'local_site': 'file://c:\\Users\\Lenovo\\Documents\\codingan\\project_m3\\dags\\gx\\uncommitted/data_docs/local_site/index.html'}

Data Docs sudah selesai dibuat dan bisa dilihat di browser.